In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import argparse
import datetime
import os
import yaml

import gymnasium as gym
import numpy as np
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

import sliding_puzzles
from callbacks import SuccessRateCallback

pygame 2.5.2 (SDL 2.28.2, Python 3.8.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:4: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if not hasattr(tensorboard, "__version__") or LooseVersion(
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/torch/utils/tensorboard/__init__.py:6: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  ) < LooseVersion("1.15"):
2024-01-21 08:28:10.556882: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-21 08:28:11.067384: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/bryan/miniconda3/envs/dreamer/lib/python3.8/site-packages/pandas/compat/nump

In [10]:
configs = {
    "run_id": datetime.datetime.now().strftime("%Y%m%d-%H%M%S"),
    # "policy": "MlpPolicy",
    "policy": "CnnPolicy",
    "total_timesteps": 1000000,
    "env_w": 2,
    "env_h": 2,
    "env_shuffle_steps": 5,
    "env_sparse_rewards": True,
    "env_variation": "image",
    "env_image_folder": "imgs/single",
    "n_envs": 32,
    # "wrapper_class": wrappers.OneHotEncodingWrapper,
    # "wrapper_class": wrappers.NormalizedObsWrapper,
    # "wrapper_class": wrappers.ImagePuzzleWrapper,
    "n_steps": 1000,
    "seed": 42,
}

In [17]:
test_env = gym.make(
    "SlidingPuzzle-v0",
    w=configs["env_w"],
    h=configs["env_h"],
    shuffle_steps=configs["env_shuffle_steps"],
    sparse_rewards=configs["env_sparse_rewards"],
    variation=configs["env_variation"],
    image_folder=configs["env_image_folder"],
)
test_env.observation_space

Box(0, 255, (128, 128, 3), uint8)

In [7]:
env = make_vec_env(
    "SlidingEnv-v0",
    n_envs=configs["n_envs"],
    seed=configs["seed"],
    env_kwargs={
        "w": configs["env_w"],
        "h": configs["env_h"],
        "shuffle_steps": configs["env_shuffle_steps"],
        "sparse_rewards": configs["env_sparse_rewards"],
        "variation": configs["env_variation"],
        "image_folder": configs["env_image_folder"],
    },
    monitor_dir=f"runs/{configs['run_id']}/logs",
    monitor_kwargs={
        "info_keywords": (
            "episode_reward",
            "is_success",
        )
    },
)

/mnt/data/Documents/sliding-puzzle/sliding-puzzle-env/sliding_puzzles/env.py:64: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  self.fig, self.ax = plt.subplots()


In [11]:
model = PPO(
    configs["policy"],
    env,
    verbose=1,
    tensorboard_log=f"runs/{configs['run_id']}",
    device="cuda",
    seed=configs["seed"],
    n_steps=configs["n_steps"],
)

Using cuda device


AssertionError: You should use NatureCNN only with images not with Box(0, 16, (4, 4), int32)
(you are probably using `CnnPolicy` instead of `MlpPolicy` or `MultiInputPolicy`)
If you are using a custom environment,
please check it using our env checker:
https://stable-baselines3.readthedocs.io/en/master/common/env_checker.html.
If you are using `VecNormalize` or already normalized channel-first images you should pass `normalize_images=False`: 
https://stable-baselines3.readthedocs.io/en/master/guide/custom_env.html

Logging to runs/20240120-113322/PPO_1


Output()

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 47.5     |
|    ep_rew_mean     | -15      |
|    success_rate    | 0        |
| time/              |          |
|    fps             | 1543     |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 3200     |
---------------------------------


-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 47.7        |
|    ep_rew_mean          | -14.5       |
|    success_rate         | 0           |
| time/                   |             |
|    fps                  | 635         |
|    iterations           | 2           |
|    time_elapsed         | 10          |
|    total_timesteps      | 6400        |
| train/                  |             |
|    approx_kl            | 0.015770169 |
|    clip_fraction        | 0.217       |
|    clip_range           | 0.2         |
|    entropy_loss         | -1.37       |
|    explained_variance   | -0.0107     |
|    learning_rate        | 0.0003      |
|    loss                 | 7.11        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0215     |
|    value_loss           | 18          |
-----------------------------------------


----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 22.1       |
|    ep_rew_mean          | 0.26       |
|    success_rate         | 0.0625     |
| time/                   |            |
|    fps                  | 545        |
|    iterations           | 3          |
|    time_elapsed         | 17         |
|    total_timesteps      | 9600       |
| train/                  |            |
|    approx_kl            | 0.01631376 |
|    clip_fraction        | 0.303      |
|    clip_range           | 0.2        |
|    entropy_loss         | -1.33      |
|    explained_variance   | 0.262      |
|    learning_rate        | 0.0003     |
|    loss                 | 13.1       |
|    n_updates            | 20         |
|    policy_gradient_loss | -0.036     |
|    value_loss           | 27.1       |
----------------------------------------


...

---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 100       |
|    ep_rew_mean          | -99.5     |
|    success_rate         | 0         |
| time/                   |           |
|    fps                  | 424       |
|    iterations           | 309       |
|    time_elapsed         | 2327      |
|    total_timesteps      | 988800    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.54e-07 |
|    explained_variance   | 0.966     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.0524    |
|    n_updates            | 3080      |
|    policy_gradient_loss | -5.23e-09 |
|    value_loss           | 0.0558    |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 100       |
|    ep_rew_mean          | -99.4     |
|    success_rate         | 0         |
| time/                   |           |
|    fps                  | 424       |
|    iterations           | 310       |
|    time_elapsed         | 2335      |
|    total_timesteps      | 992000    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.72e-07 |
|    explained_variance   | 0.948     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.0268    |
|    n_updates            | 3090      |
|    policy_gradient_loss | -3.28e-09 |
|    value_loss           | 0.0746    |
---------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 100       |
|    ep_rew_mean          | -99.5     |
|    success_rate         | 0         |
| time/                   |           |
|    fps                  | 424       |
|    iterations           | 311       |
|    time_elapsed         | 2342      |
|    total_timesteps      | 995200    |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.01e-07 |
|    explained_variance   | 0.951     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.00141   |
|    n_updates            | 3100      |
|    policy_gradient_loss | -5.88e-09 |
|    value_loss           | 0.0154    |
---------------------------------------


--------------------------------------
| rollout/                |          |
|    ep_len_mean          | 100      |
|    ep_rew_mean          | -99.5    |
|    success_rate         | 0        |
| time/                   |          |
|    fps                  | 424      |
|    iterations           | 312      |
|    time_elapsed         | 2350     |
|    total_timesteps      | 998400   |
| train/                  |          |
|    approx_kl            | 0.0      |
|    clip_fraction        | 0        |
|    clip_range           | 0.2      |
|    entropy_loss         | -7.3e-07 |
|    explained_variance   | 0.972    |
|    learning_rate        | 0.0003   |
|    loss                 | 0.0974   |
|    n_updates            | 3110     |
|    policy_gradient_loss | -4.1e-09 |
|    value_loss           | 0.0877   |
--------------------------------------


---------------------------------------
| rollout/                |           |
|    ep_len_mean          | 100       |
|    ep_rew_mean          | -99.5     |
|    success_rate         | 0         |
| time/                   |           |
|    fps                  | 424       |
|    iterations           | 313       |
|    time_elapsed         | 2357      |
|    total_timesteps      | 1001600   |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -5.59e-07 |
|    explained_variance   | 0.995     |
|    learning_rate        | 0.0003    |
|    loss                 | 0.00497   |
|    n_updates            | 3120      |
|    policy_gradient_loss | -1.74e-10 |
|    value_loss           | 0.04      |
---------------------------------------
